In [1]:
# hide

%load_ext nb_black

<IPython.core.display.Javascript object>

In [10]:
import json

from pathlib import Path

from will_it_saturate.core import Benchmark

<IPython.core.display.Javascript object>

# Create wrk JSON

In [11]:
byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte

benchmark = Benchmark(bandwidth=bandwidth, duration=3)

# file_sizes = [10 ** 7, 10 ** 6]
file_sizes = [10 ** 7, 10 ** 6, 10 ** 5]
# file_sizes = [10 ** 7]
benchmark.create_rows(file_sizes)

<IPython.core.display.Javascript object>

In [12]:
def create_url_list(benchmark_row):
    template = {
        "method": "GET",
    }
    urls = []
    for bf in benchmark_row.files:
        urls.append({**template, "path": bf.path})
    return urls

<IPython.core.display.Javascript object>

In [19]:
for br in benchmark.rows:
    urls = create_url_list(br)
    print(len(urls))
    json_path = Path(br.data_root) / f"{br.base_path}.json"
    print(json_path)
    with json_path.open("w") as f:
        f.write(json.dumps(urls))

38
data/10000000_3_125000000.json
375
data/1000000_3_125000000.json
3750
data/100000_3_125000000.json


<IPython.core.display.Javascript object>

# Create wrk Lua

In [38]:
def cerate_url_string(benchmark_row):
    urls = []
    for bf in benchmark_row.files:
        urls.append(f'    {{path = "/{bf.path}"}},')
    return "\n".join(urls)

<IPython.core.display.Javascript object>

In [39]:
requests_head = "requests = {"
requests_tail = "}"

lua_body = """
print(requests[1])

if #requests <= 0 then
  print("multiplerequests: No requests found.")
  os.exit()
end

print("multiplerequests: Found " .. #requests .. " requests")

counter = 1
request = function()
  -- Get the next requests array element
  local request_object = requests[counter]

  -- Increment the counter
  counter = counter + 1

  -- If the counter is longer than the requests array length then reset it
  if counter > #requests then
    counter = 1
  end

  -- Return the request object with the current URL path
  return wrk.format(request_object.method, request_object.path, request_object.headers, request_object.body)
end
"""

<IPython.core.display.Javascript object>

In [40]:
for num, br in enumerate(benchmark.rows):
    urls = cerate_url_string(br)
    lua = "\n".join([requests_head, urls, requests_tail, lua_body])
    with Path(f"wrk{num}.lua").open("w") as f:
        f.write(lua)

<IPython.core.display.Javascript object>